In [ ]:

import sys
sys.path.append(r"C:\Users\Talha Ali\Desktop\machine learning labs\timeseires")

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model

import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [11]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [13]:
def MLP():
    model = Sequential()
    model.add(Flatten(input_shape=(time_steps , num_features)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    return model

In [15]:
model1 = MLP()
model1.summary()

C:\Users\PMLS\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 504)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │          16,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,193 (63.25 KB)

 Trainable params: 16,193 (63.25 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Talha Ali\Desktop\machine learning labs'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [21]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [23]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =MLP()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [ ]:
import os
path_dataset =r'C:\Users\Talha Ali\Desktop\machine learning labs'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [27]:
df_te.head(1)

,aep,Is_holiday1,Is_holiday2,Is_Weekend1,Is_Weekend2,sin_month,cos_month,sin_week,cos_week,sin_hour,...,sin_wintert,cos_wintert,sin_springt,cos_springt,sin_summert,cos_summert,sin_fallt,cos_fallt,sin_year_dayt,cos_year_dayt
0,0.518532,1.0,0.0,1.0,0.0,1.0,6.123234e-17,0.866025,-0.5,-0.707107,...,0.0,1.0,1.0,6.123234e-17,0.0,1.0,0.0,1.0,0.956235,0.2926


In [29]:
test_set[0]

array([ 5.18532385e-01,  1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  6.12323400e-17,  8.66025404e-01,
       -5.00000000e-01, -7.07106781e-01, -7.07106781e-01,  0.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  6.12323400e-17,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  9.56234827e-01,
        2.92600336e-01])

In [31]:
time_steps=2
num_features=21

In [33]:
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=0)

In [35]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02]])

In [37]:
train_y[0]

array([], dtype=float64)

In [39]:
time_steps =24
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=1,target_len=2)

In [41]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.26232372e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e

In [43]:
train_y[0]

array([1., 1.])

In [45]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.4148116111755371 sec


In [49]:
epochs = 30
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                    verbose = verbose)

Epoch 1/30
2617/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0174 - mae: 0.0174 - mape: 44.0588
Epoch 1: val_loss improved from 0.01738 to 0.01672, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0174 - mae: 0.0174 - mape: 49.2601 - val_loss: 0.0167 - val_mae: 0.0167 - val_mape: 7.3132
Epoch 2/30
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0165 - mae: 0.0165 - mape: 2697.1426
Epoch 2: val_loss improved from 0.01672 to 0.01613, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\\E1-cp-0002-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0165 - mae: 0.0165 - mape: 2694.8789 - val_loss: 0.0161 - val_mae: 0.0161 - val_mape: 7.1427
Epoch 3/30
2642/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0160 - mae: 0.0160 - mape: 449.9112
Epoch 3: val_loss did not improve from 0.01613
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0160 - mae: 0.0160 - mape: 449.7070 - val_loss: 0.0170 - val_mae: 0.0170 - val_mape: 7.2144
Epoch 4/30
2633/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0156 - mae: 0.0156 - mape: 394.1133
Epoch 4: val_loss did not improve from 0.01613
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0156 - mae: 0.0156 - mape: 392.8590 - val_loss: 0.0164 - val_mae: 0.0164 - val_mape: 7.4616
Epoch 5/30
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0150 - mae: 0.0150 - mape: 324.2248
Epoch 5: val_loss improved from 0.01613 to 0.01380, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0150 - mae: 0.0150 - mape: 323.8373 - val_loss: 0.0138 - val_mae: 0.0138 - val_mape: 6.0655
Epoch 6/30
2628/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0150 - mae: 0.0150 - mape: 688.9628
Epoch 6: val_loss did not improve from 0.01380
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0150 - mae: 0.0150 - mape: 685.3497 - val_loss: 0.0142 - val_mae: 0.0142 - val_mape: 5.9711
Epoch 7/30
2629/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0149 - mae: 0.0149 - mape: 254.1535
Epoch 7: val_loss improved from 0.01380 to 0.01378, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\\E1-cp-0007-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0149 - mae: 0.0149 - mape: 255.2169 - val_loss: 0.0138 - val_mae: 0.0138 - val_mape: 6.1489
Epoch 8/30
2630/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0147 - mae: 0.0147 - mape: 1260.1628
Epoch 8: val_loss improved from 0.01378 to 0.01349, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\\E1-cp-0008-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0147 - mae: 0.0147 - mape: 1252.0160 - val_loss: 0.0135 - val_mae: 0.0135 - val_mape: 6.2974
Epoch 9/30
2634/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0143 - mae: 0.0143 - mape: 385.4995
Epoch 9: val_loss did not improve from 0.01349
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0143 - mae: 0.0143 - mape: 385.4050 - val_loss: 0.0151 - val_mae: 0.0151 - val_mape: 6.2730
Epoch 10/30
2621/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0141 - mae: 0.0141 - mape: 24.2171
Epoch 10: val_loss did not improve from 0.01349
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0141 - mae: 0.0141 - mape: 25.7772 - val_loss: 0.0135 - val_mae: 0.0135 - val_mape: 5.9935
Epoch 11/30
2628/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0140 - mae: 0.0140 - mape: 455.4535
Epoch 11: val_loss did not improve from 0.01349
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0140 - mae: 0.0140 - mape: 453.3624 - val_loss: 0.0171 - val_mae: 0.

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0135 - mae: 0.0135 - mape: 68.5579 - val_loss: 0.0132 - val_mae: 0.0132 - val_mape: 5.9860
Epoch 15/30
2629/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0131 - mae: 0.0131 - mape: 14.5531
Epoch 15: val_loss improved from 0.01320 to 0.01203, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\\E1-cp-0015-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0131 - mae: 0.0131 - mape: 19.6766 - val_loss: 0.0120 - val_mae: 0.0120 - val_mape: 5.6438
Epoch 16/30
2621/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0131 - mae: 0.0131 - mape: 833.0591
Epoch 16: val_loss did not improve from 0.01203
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0131 - mae: 0.0131 - mape: 827.5319 - val_loss: 0.0141 - val_mae: 0.0141 - val_mape: 6.1513
Epoch 17/30
2614/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0131 - mae: 0.0131 - mape: 4.3108
Epoch 17: val_loss did not improve from 0.01203
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0131 - mae: 0.0131 - mape: 9.6771 - val_loss: 0.0136 - val_mae: 0.0136 - val_mape: 6.3748
Epoch 18/30
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0130 - mae: 0.0130 - mape: 396.6725
Epoch 18: val_loss did not improve from 0.01203
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0130 - mae: 0.0130 - mape: 396.5338 - val_loss: 0.0123 - val_mae: 0.01

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0126 - mae: 0.0126 - mape: 100.9790 - val_loss: 0.0115 - val_mae: 0.0115 - val_mape: 5.0339
Epoch 21/30
2643/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0126 - mae: 0.0126 - mape: 118.7674
Epoch 21: val_loss did not improve from 0.01146
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0126 - mae: 0.0126 - mape: 120.1510 - val_loss: 0.0138 - val_mae: 0.0138 - val_mape: 6.1919
Epoch 22/30
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0125 - mae: 0.0125 - mape: 164.3900
Epoch 22: val_loss did not improve from 0.01146
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0125 - mae: 0.0125 - mape: 164.5240 - val_loss: 0.0123 - val_mae: 0.0123 - val_mape: 5.5259
Epoch 23/30
2630/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0124 - mae: 0.0124 - mape: 29.8228
Epoch 23: val_loss improved from 0.01146 to 0.01137, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\\E1-cp-0023-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0124 - mae: 0.0124 - mape: 32.8274 - val_loss: 0.0114 - val_mae: 0.0114 - val_mape: 4.9056
Epoch 24/30
2627/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0124 - mae: 0.0124 - mape: 791.6733
Epoch 24: val_loss did not improve from 0.01137
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0124 - mae: 0.0124 - mape: 787.5219 - val_loss: 0.0156 - val_mae: 0.0156 - val_mape: 7.4412
Epoch 25/30
2626/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0125 - mae: 0.0125 - mape: 323.6113
Epoch 25: val_loss did not improve from 0.01137
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0125 - mae: 0.0125 - mape: 325.4000 - val_loss: 0.0137 - val_mae: 0.0137 - val_mape: 5.8589
Epoch 26/30
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0124 - mae: 0.0124 - mape: 1968.9717
Epoch 26: val_loss did not improve from 0.01137
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0124 - mae: 0.0124 - mape: 1967.8571 - val_loss: 0.0142 - val_mae

In [ ]:

model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labs\E1-cp-0023-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})


y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step
Mean Absolute Error (MAE): 183.65
Median Absolute Error (MedAE): 154.52
Mean Squared Error (MSE): 54497.9
Root Mean Squared Error (RMSE): 233.45
Mean Absolute Percentage Error (MAPE): 1.25 %
Median Absolute Percentage Error (MDAPE): 1.07 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


# Fine Tuning

In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labslab8\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Talha Ali\Desktop\machine learning labs\lab7\E1-cp-0023-loss0.01.h5'
start_epoch= 20

In [ ]:
import os
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, InputLayer
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K

# Define your model builder function (replace PC.build)
def build_model(time_steps, num_features, reg=0.0005):
    model = Sequential()
    model.add(InputLayer(input_shape=(time_steps, num_features)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(1))  # Adjust output layer size according to your task
    return model


# Path to save/load your model checkpoint
model_path = r'C:\Users\Talha Ali\Desktop\machine learning labs\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'

# Construct the callback to save only the *best* model based on validation loss
EpochCheckpoint1 = ModelCheckpoint(
    model_path,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1]

# Variable 'model' for holding your model object
model = None

# Check if checkpoint file exists
if not os.path.exists(model_path):
    print("[INFO] No checkpoint found. Compiling new model...")
    model = build_model(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(learning_rate=1e-3)
    model.compile(loss="mae", optimizer=opt, metrics=["mae", "mape"])
else:
    print(f"[INFO] Loading model from {model_path} ...")
    model = load_model(model_path)

    # Update the learning rate
    print(f"[INFO] Old learning rate: {K.get_value(model.optimizer.lr)}")
    K.set_value(model.optimizer.lr, 1e-4)
    print(f"[INFO] New learning rate: {K.get_value(model.optimizer.lr)}")

# Now you can train your model with callbacks
# Example:
# model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=..., callbacks=callbacks)


[INFO] No checkpoint found. Compiling new model...


C:\Users\PMLS\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [65]:
epochs = 20
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/20
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0423 - mae: 0.0423 - mape: 237.9978
Epoch 1: val_loss improved from inf to 0.01378, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0001-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0423 - mae: 0.0423 - mape: 238.0196 - val_loss: 0.0138 - val_mae: 0.0138 - val_mape: 6.0630
Epoch 2/20
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0130 - mae: 0.0130 - mape: 85.9563
Epoch 2: val_loss improved from 0.01378 to 0.00916, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0130 - mae: 0.0130 - mape: 86.0859 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 4.0743
Epoch 3/20
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0100 - mae: 0.0100 - mape: 129.9615
Epoch 3: val_loss did not improve from 0.00916
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0100 - mae: 0.0100 - mape: 129.8133 - val_loss: 0.0095 - val_mae: 0.0095 - val_mape: 3.9440
Epoch 4/20
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0093 - mae: 0.0093 - mape: 335.5127
Epoch 4: val_loss improved from 0.00916 to 0.00830, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0093 - mae: 0.0093 - mape: 334.8712 - val_loss: 0.0083 - val_mae: 0.0083 - val_mape: 3.7727
Epoch 5/20
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0086 - mae: 0.0086 - mape: 8.2427
Epoch 5: val_loss improved from 0.00830 to 0.00807, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0086 - mae: 0.0086 - mape: 8.5244 - val_loss: 0.0081 - val_mae: 0.0081 - val_mape: 3.4262
Epoch 6/20
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0083 - mae: 0.0083 - mape: 18.6127
Epoch 6: val_loss improved from 0.00807 to 0.00769, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0006-loss0.01.h5


...

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0079 - mae: 0.0079 - mape: 32.5964 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 3.2573
Epoch 8/20
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0076 - mae: 0.0076 - mape: 19.7469
Epoch 8: val_loss improved from 0.00767 to 0.00725, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0008-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0076 - mae: 0.0076 - mape: 19.9140 - val_loss: 0.0072 - val_mae: 0.0072 - val_mape: 3.2303
Epoch 9/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0074 - mae: 0.0074 - mape: 5.4551
Epoch 9: val_loss did not improve from 0.00725
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0074 - mae: 0.0074 - mape: 5.4558 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.7782
Epoch 10/20
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0072 - mae: 0.0072 - mape: 4.5572
Epoch 10: val_loss did not improve from 0.00725
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0072 - mae: 0.0072 - mape: 4.6385 - val_loss: 0.0086 - val_mae: 0.0086 - val_mape: 3.8581
Epoch 11/20
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0070 - mae: 0.0070 - mape: 72.6693
Epoch 11: val_loss improved from 0.00725 to 0.00657, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0011-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0070 - mae: 0.0070 - mape: 72.8811 - val_loss: 0.0066 - val_mae: 0.0066 - val_mape: 2.9127
Epoch 12/20
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0070 - mae: 0.0070 - mape: 51.8051
Epoch 12: val_loss improved from 0.00657 to 0.00640, saving model to C:\Users\PMLS\shahid (ML-lab)\lab7\E2-cp-0012-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 38s 8ms/step - loss: 0.0070 - mae: 0.0070 - mape: 51.7886 - val_loss: 0.0064 - val_mae: 0.0064 - val_mape: 2.6120
Epoch 13/20
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0067 - mae: 0.0067 - mape: 129.9682
Epoch 13: val_loss did not improve from 0.00640
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0067 - mae: 0.0067 - mape: 129.9645 - val_loss: 0.0068 - val_mae: 0.0068 - val_mape: 2.8448
Epoch 14/20
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0068 - mae: 0.0068 - mape: 25.7263
Epoch 14: val_loss did not improve from 0.00640
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0068 - mae: 0.0068 - mape: 25.8537 - val_loss: 0.0066 - val_mae: 0.0066 - val_mape: 2.7657
Epoch 15/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0067 - mae: 0.0067 - mape: 26.9499
Epoch 15: val_loss did not improve from 0.00640
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0067 - mae: 0.0067 - mape: 26.9816 - val_loss: 0.0066 - val_mae: 

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0066 - mae: 0.0066 - mape: 47.4932 - val_loss: 0.0061 - val_mae: 0.0061 - val_mape: 2.5736
Epoch 17/20
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0065 - mae: 0.0065 - mape: 35.1273
Epoch 17: val_loss did not improve from 0.00611
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0065 - mae: 0.0065 - mape: 35.1861 - val_loss: 0.0082 - val_mae: 0.0082 - val_mape: 3.4926
Epoch 18/20
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0065 - mae: 0.0065 - mape: 9.8925
Epoch 18: val_loss did not improve from 0.00611
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0065 - mae: 0.0065 - mape: 9.9597 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.4738
Epoch 19/20
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0064 - mae: 0.0064 - mape: 4.4320
Epoch 19: val_loss did not improve from 0.00611
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0064 - mae: 0.0064 - mape: 4.4309 - val_loss: 0.0072 - val_mae: 0.0072

In [ ]:
model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labs\lab7\E1-cp-0059-loss0.01.h5',custom_objects={'mae': mean_absolute_error})


y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step
Mean Absolute Error (MAE): 161.9
Median Absolute Error (MedAE): 135.89
Mean Squared Error (MSE): 42708.55
Root Mean Squared Error (RMSE): 206.66
Mean Absolute Percentage Error (MAPE): 1.11 %
Median Absolute Percentage Error (MDAPE): 0.94 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
